In [ ]:
from pyspark.sql import SparkSession, HiveContext
spark = SparkSession.builder.appName('Ops').getOrCreate()

from pyspark.sql.types import StructType, StructField
from pyspark.sql.types import DoubleType, IntegerType, StringType



In [15]:
import pyspark
import pandas as pd

In [23]:
wb_out= spark.read.option("header","true").option("delimiter",",").option("inferSchema", "true").format("csv").load("/home/qx816/notebooks/Wb_may8/data/model_input/raw/weibull_b/build.csv")

wb_out_1= spark.read.option("header","true").option("delimiter",",").option("inferSchema", "true").format("csv").load("/home/qx816/notebooks/Wb_may8/data/model_input/raw/weibull_b/claims.csv")


In [24]:
failm_ou1= spark.read.option("header","true").option("delimiter",",").option("inferSchema", "true").format("csv").load("/home/qx816/notebooks/Wb_may8/data/model_input/weibull_b/incidents_csv/failcode_input.csv")

fault_ou2= spark.read.option("header","true").option("delimiter",",").option("inferSchema", "true").format("csv").load("/home/qx816/notebooks/Wb_may8/data/model_input/weibull_b/incidents_csv/faultcode_input.csv")


In [33]:
CENS_B= spark.read.option("header","true").option("delimiter",",").option("inferSchema", "true").format("csv").load("/home/qx816/notebooks/ap_25/data/model_output/weibull_b/censor_files/2018-01-04_IRCL_WARRANTY_B6.7 2017_AUTO_2017_02_0_0_0_0_0_0_CENSOR_DATA_b.csv")

CENS_X= spark.read.option("header","true").option("delimiter",",").option("inferSchema", "true").format("csv").load("/home/qx816/notebooks/ap_25/data/model_output/weibull_x/censor_files/2018-01-04_THAS_WARRANTY_X15 1 2017_AUTO_2017_05_0_0_0_0_0_0_CENSOR_DATA_x.csv")


In [4]:
failc_b= spark.read.option("header","true").option("delimiter",",").option("inferSchema", "true").format("csv").load("/home/qx816/notebooks/ap_25/data/model_input/weibull_b/incidents_csv/failcode_input.csv")
fail_x = spark.read.option("header","true").option("delimiter",",").option("inferSchema", "true").format("csv").load("/home/qx816/notebooks/ap_25/data/model_input/weibull_x/incidents_csv/failcode_input.csv")


In [5]:
failc_b.printSchema()
fail_x.printSchema()


root
 |-- REL_ENGINE_NAME_DESC: string (nullable = true)
 |-- ENGINE_SERIAL_NUM: integer (nullable = true)
 |-- ENGINE_MILES: double (nullable = true)
 |-- PROGRAM_GROUP_NAME: string (nullable = true)
 |-- EARLIEST_INDICATION_DATE: timestamp (nullable = true)
 |-- REL_USER_APPL_DESC: string (nullable = true)
 |-- BUILD_DATE: timestamp (nullable = true)
 |-- NET_AMOUNT: double (nullable = true)
 |-- FAILURE_DATE: timestamp (nullable = true)
 |-- CODE: string (nullable = true)
 |-- CLAIM_DATE: timestamp (nullable = true)

root
 |-- REL_ENGINE_NAME_DESC: string (nullable = true)
 |-- ENGINE_SERIAL_NUM: integer (nullable = true)
 |-- ENGINE_MILES: double (nullable = true)
 |-- PROGRAM_GROUP_NAME: string (nullable = true)
 |-- EARLIEST_INDICATION_DATE: timestamp (nullable = true)
 |-- REL_USER_APPL_DESC: string (nullable = true)
 |-- BUILD_DATE: timestamp (nullable = true)
 |-- NET_AMOUNT: double (nullable = true)
 |-- FAILURE_DATE: timestamp (nullable = true)
 |-- CODE: string (nullable = 

In [6]:
CENS_B.printSchema()
CENS_X.printSchema()

NameError: name 'CENS_B' is not defined

In [7]:

sqlContext.registerDataFrameAsTable(failc_b, "failc_b")
sqlContext.registerDataFrameAsTable(fail_x, "failc_x")

In [30]:
cd = sqlContext.sql("select distinct(REL_ENGINE_NAME_DESC) from failc_x")

cd.show()
cd.toPandas()



+--------------------+
|REL_ENGINE_NAME_DESC|
+--------------------+
|          X15 3 2017|
|          X15 1 2017|
|           ISX1 2013|
|           ISX2 2013|
|           ISX3 2013|
+--------------------+



,REL_ENGINE_NAME_DESC
0,X15 3 2017
1,X15 1 2017
2,ISX1 2013
3,ISX2 2013
4,ISX3 2013


In [27]:
lst = cd["REL_ENGINE_NAME_DESC"].tolist()

print(lst)




TypeError: 'Column' object is not callable

In [ ]:
sk = 'X15 1 2017'
print(sk)

In [58]:
ck= sqlContext.sql("select max(BUILD_DATE) as build, max(FAILURE_DATE) as fail, max(CLAIM_DATE) as claim, min(BUILD_DATE) as min_build, min(FAILURE_DATE) as min_fail, min(CLAIM_DATE) as min_claim from failc_b ")
ck1= sqlContext.sql("select  max(BUILD_DATE) as build, max(FAILURE_DATE) as fail, max(CLAIM_DATE), min(BUILD_DATE) as build, min(FAILURE_DATE) as fail, min(CLAIM_DATE) as claim from failc_x ")

ck.show()
ck1.show()


+-------------------+-------------------+-------------------+-------------------+-------------------+-------------------+
|              build|               fail|              claim|          min_build|           min_fail|          min_claim|
+-------------------+-------------------+-------------------+-------------------+-------------------+-------------------+
|2017-11-30 00:00:00|2018-01-04 00:00:00|2017-12-29 00:00:00|2017-01-02 00:00:00|2017-01-17 00:00:00|2017-01-26 00:00:00|
+-------------------+-------------------+-------------------+-------------------+-------------------+-------------------+

+-------------------+-------------------+-------------------+-------------------+-------------------+-------------------+
|              build|               fail|    max(CLAIM_DATE)|              build|               fail|              claim|
+-------------------+-------------------+-------------------+-------------------+-------------------+-------------------+
|2017-12-11 00:00:00|20

In [66]:
ck= sqlContext.sql("select max(BUILD_DATE) as max, min(BUILD_DATE) as min from failc_b where BUILD_DATE between '2017-02-01' and '2017-02-31' ")
ck.show()

+-------------------+-------------------+
|                max|                min|
+-------------------+-------------------+
|2017-02-28 00:00:00|2017-02-01 00:00:00|
+-------------------+-------------------+



In [73]:
ck= sqlContext.sql("select distinct(ENGINE_SERIAL_NUM) from failc_b where code == 'IRCL' and PROGRAM_GROUP_NAME == 'WARRANTY' and REL_USER_APPL_DESC == 'AUTO' and BUILD_DATE between '2017-02-01' and '2017-02-30' and REL_ENGINE_NAME_DESC == 'B6.7 2017' ")
ck1= sqlContext.sql("select distinct(ENGINE_SERIAL_NUM) from failc_x where code == 'THAS' and PROGRAM_GROUP_NAME == 'WARRANTY' and REL_USER_APPL_DESC == 'AUTO' and BUILD_DATE between '2017-05-01' and '2017-05-31' and REL_ENGINE_NAME_DESC == 'X15 1 2017' ")

ck.show()
ck.count()


+-----------------+
|ENGINE_SERIAL_NUM|
+-----------------+
|         74099500|
|         74097022|
|         74099820|
|         74101173|
+-----------------+



4

In [78]:
ck= sqlContext.sql("select sum(NET_AMOUNT) as SUM,ENGINE_SERIAL_NUM from failc_b where code == 'IRCL' and PROGRAM_GROUP_NAME == 'WARRANTY' and REL_USER_APPL_DESC == 'AUTO' and BUILD_DATE between '2017-02-01' and '2017-02-30' and REL_ENGINE_NAME_DESC == 'B6.7 2017' group by ENGINE_SERIAL_NUM ")
ck1= sqlContext.sql("select sum(NET_AMOUNT) as SUM,ENGINE_SERIAL_NUM from failc_x where code == 'THAS' and PROGRAM_GROUP_NAME == 'WARRANTY' and REL_USER_APPL_DESC == 'AUTO' and BUILD_DATE between '2017-05-01' and '2017-05-31' and REL_ENGINE_NAME_DESC == 'X15 1 2017' group by ENGINE_SERIAL_NUM")

ck.show()
ck1.show()


+-------+-----------------+
|    SUM|ENGINE_SERIAL_NUM|
+-------+-----------------+
|1119.22|         74099500|
|6538.28|         74097022|
|1202.31|         74099820|
|1026.85|         74101173|
+-------+-----------------+

+--------+-----------------+
|     SUM|ENGINE_SERIAL_NUM|
+--------+-----------------+
|15015.37|         79982161|
|10162.91|         79980728|
| 2753.67|         79983796|
|23909.62|         79982134|
|18748.07|         79980350|
| 2396.02|         79978882|
| 2616.46|         79981853|
|  176.49|         79981432|
|19530.71|         79982806|
| 8106.01|         79981115|
|24340.27|         79979925|
| 8204.47|         79981593|
| 8531.37|         79982185|
|49919.25|         79983548|
| 4270.89|         79981438|
+--------+-----------------+



In [76]:

sqlContext.registerDataFrameAsTable(ck, "ck")
sqlContext.registerDataFrameAsTable(ck1, "ck1")

In [77]:
cdk = sqlContext.sql("select sum(SUM) from ck")
cdk1 = sqlContext.sql("select sum(SUM) from ck1")

cdk.show()
cdk1.show()

+--------+
|sum(SUM)|
+--------+
| 9886.66|
+--------+

+------------------+
|          sum(SUM)|
+------------------+
|198681.58000000002|
+------------------+



In [74]:
ck1.show()

ck1.count()

+-----------------+
|ENGINE_SERIAL_NUM|
+-----------------+
|         79982161|
|         79980728|
|         79983796|
|         79982134|
|         79980350|
|         79978882|
|         79981853|
|         79981432|
|         79982806|
|         79981115|
|         79979925|
|         79981593|
|         79982185|
|         79983548|
|         79981438|
+-----------------+



15

In [34]:

sqlContext.registerDataFrameAsTable(CENS_B, "CENS_B")
sqlContext.registerDataFrameAsTable(CENS_X, "CENS_X")

In [40]:
ck= sqlContext.sql("select max(BUILD_DATE) from CENS_B")
ck1= sqlContext.sql("select max(BUILD_DATE) from CENS_X")

ck.show()
ck1.show()


+-------------------+
|    max(BUILD_DATE)|
+-------------------+
|2017-02-28 00:00:00|
+-------------------+

+-------------------+
|    max(BUILD_DATE)|
+-------------------+
|2017-05-31 00:00:00|
+-------------------+



In [41]:
ck= sqlContext.sql("select distinct(serial) from CENS_B")


ck.show(20)
ck.count()

+--------+
|  serial|
+--------+
|74098520|
|74099780|
|74105582|
|74101796|
|74100999|
|74106937|
|74106561|
|74105859|
|74101896|
|74095529|
|74107426|
|74096491|
|74100208|
|74099808|
|74100016|
|74098058|
|74101810|
|74103840|
|74107805|
|74097940|
+--------+
only showing top 20 rows



5283

In [42]:
ck= sqlContext.sql("select distinct(serial) from CENS_X")


ck.show(20)
ck.count()

+--------+
|  serial|
+--------+
|79983591|
|79983218|
|79980802|
|79980257|
|79982126|
|79983867|
|79984362|
|79984022|
|79978946|
|79984410|
|79984306|
|79982368|
|79979059|
|79979948|
|79984326|
|79982145|
|79983472|
|79980799|
|79981830|
|79983562|
+--------+
only showing top 20 rows



3119

In [28]:
failm_ou1.printSchema()
failm_ou1.count()

root
 |-- REL_ENGINE_NAME_DESC: string (nullable = true)
 |-- ENGINE_SERIAL_NUM: integer (nullable = true)
 |-- ENGINE_MILES: double (nullable = true)
 |-- PROGRAM_GROUP_NAME: string (nullable = true)
 |-- EARLIEST_INDICATION_DATE: timestamp (nullable = true)
 |-- REL_USER_APPL_DESC: string (nullable = true)
 |-- BUILD_DATE: timestamp (nullable = true)
 |-- NET_AMOUNT: double (nullable = true)
 |-- FAILURE_DATE: timestamp (nullable = true)
 |-- CODE: string (nullable = true)
 |-- CLAIM_DATE: timestamp (nullable = true)



335031

In [29]:
fault_ou2.printSchema()
fault_ou2.count()

root
 |-- REL_ENGINE_NAME_DESC: string (nullable = true)
 |-- ENGINE_SERIAL_NUM: integer (nullable = true)
 |-- ENGINE_MILES: double (nullable = true)
 |-- PROGRAM_GROUP_NAME: string (nullable = true)
 |-- EARLIEST_INDICATION_DATE: timestamp (nullable = true)
 |-- REL_USER_APPL_DESC: string (nullable = true)
 |-- BUILD_DATE: timestamp (nullable = true)
 |-- NET_AMOUNT: double (nullable = true)
 |-- FAILURE_DATE: timestamp (nullable = true)
 |-- CODE: integer (nullable = true)
 |-- CLAIM_DATE: timestamp (nullable = true)



206969

In [30]:

sqlContext.registerDataFrameAsTable(failm_ou1, "failm_ou1")
sqlContext.registerDataFrameAsTable(fault_ou2, "fault_ou2")

In [31]:
ck = sqlContext.sql("select max(BUILD_DATE) as Max, min(BUILD_DATE) as Min from failm_ou1")

ck.show()

+-------------------+-------------------+
|                Max|                Min|
+-------------------+-------------------+
|2019-03-29 00:00:00|2011-02-21 00:00:00|
+-------------------+-------------------+



In [32]:
ck = sqlContext.sql("select max(BUILD_DATE) as Max, min(BUILD_DATE) as Min from fault_ou2")

ck.show()

+-------------------+-------------------+
|                Max|                Min|
+-------------------+-------------------+
|2019-04-11 00:00:00|2011-04-20 00:00:00|
+-------------------+-------------------+



In [ ]:
#om_out_52= spark.read.option("header","true").option("delimiter",",").option("inferSchema", "true").format("csv").load("/data/data/om/OM_output/om_model_output/OM_MODEL_OUTPUT_2019-05-02.csv")


In [ ]:
#om_out_52.printSchema()

In [ ]:
#sqlContext.registerDataFrameAsTable(om_out_52, "om_out_52_df")

In [25]:

wb_out.printSchema()
wb_out.count()

root
 |-- ESN: string (nullable = true)
 |-- ENG_NAME: string (nullable = true)
 |-- BUILD_YEAR: string (nullable = true)
 |-- CPL_NUM: string (nullable = true)
 |-- DESIGN_CONFIG_NUM: string (nullable = true)
 |-- DESIGN_HSP_NUM: integer (nullable = true)
 |-- DESIGN_RPM_NUM: integer (nullable = true)
 |-- ECM_PART_NO: integer (nullable = true)
 |-- EMISSIONS_FAMILY_CODE: string (nullable = true)
 |-- ENGINE_GROUP_DESC: string (nullable = true)
 |-- EPA_NAME: string (nullable = true)
 |-- MKTG_CONFIG_NUM: string (nullable = true)
 |-- MKTG_HSP_NUM: integer (nullable = true)
 |-- MKTG_RPM_NUM: integer (nullable = true)
 |-- OEM_NAME: string (nullable = true)
 |-- SERVICE_ENGINE_MODEL: string (nullable = true)
 |-- SHOP_ORDER_NUM: integer (nullable = true)
 |-- TRUCK_MODEL: string (nullable = true)
 |-- ENGINE_SERIAL_NUM: integer (nullable = true)
 |-- BUILD_DATE: timestamp (nullable = true)
 |-- BUILD_MONTH: string (nullable = true)
 |-- USER_APPL_CODE: integer (nullable = true)
 |-- D

31325545

In [26]:
wb_out_1.printSchema()
wb_out_1.count()

root
 |-- ENG_NAME: string (nullable = true)
 |-- ENGINE_SERIAL_NUM: integer (nullable = true)
 |-- BUILD_DATE: timestamp (nullable = true)
 |-- BUILD_MONTH: string (nullable = true)
 |-- FAIL_CODE: string (nullable = true)
 |-- IN_SERVICE_DATE: timestamp (nullable = true)
 |-- FAILURE_DATE: timestamp (nullable = true)
 |-- USER_APPL_CODE: integer (nullable = true)
 |-- ENGINE_MILES: double (nullable = true)
 |-- ENGINE_HRS: double (nullable = true)
 |-- CLAIM_DATE: timestamp (nullable = true)
 |-- PROGRAM_GROUP_CODE: integer (nullable = true)
 |-- PROGRAM_GROUP_NAME: string (nullable = true)
 |-- PAYMENT_DATE: timestamp (nullable = true)
 |-- MATERIALS_AMOUNT: double (nullable = true)
 |-- LABOR_HOURS: double (nullable = true)
 |-- MARKUP_AMOUNT: double (nullable = true)
 |-- REPAIR_LABOR_AMOUNT: double (nullable = true)
 |-- TRAVEL_LABOR_AMOUNT: double (nullable = true)
 |-- UNDETAILED_PARTS_AMOUNT: double (nullable = true)
 |-- INVOICE_AMOUNT: double (nullable = true)
 |-- NET_AMOUN

389061

In [27]:
sqlContext.registerDataFrameAsTable(wb_out, "wb_out")
sqlContext.registerDataFrameAsTable(wb_out_1, "wb_out_1")


In [20]:
cd=sqlContext.sql("select max(BUILD_DATE) as Max,min(BUILD_DATE) as Min from wb_out")

cd.show()
cd.count()

+-------------------+-------------------+
|                Max|                Min|
+-------------------+-------------------+
|2019-04-30 00:00:00|2010-09-03 00:00:00|
+-------------------+-------------------+



1

In [21]:
cd=sqlContext.sql("select max(BUILD_DATE) as Max,min(BUILD_DATE) as Min from wb_out_1")

cd.show()
cd.count()

+-------------------+-------------------+
|                Max|                Min|
+-------------------+-------------------+
|2019-03-29 00:00:00|2011-02-18 00:00:00|
+-------------------+-------------------+



1

In [22]:
cd=sqlContext.sql("select max(PAYMENT_DATE) as Max,min(PAYMENT_DATE) as Min from wb_out_1")

cd.show()
cd.count()

+-------------------+-------------------+
|                Max|                Min|
+-------------------+-------------------+
|2019-05-02 00:00:00|2011-07-05 00:00:00|
+-------------------+-------------------+



1

In [44]:
cdf = cd.toPandas()


cdf.to_csv('Wb_L_file_May_2_Gold.csv')

In [27]:
co=sqlContext.sql("select OM_CRRT_PRIORITY_SCORE from om_out_df where om_algo == 'MA_Ribbon' and cast(OM_CRRT_PRIORITY_SCORE as int)<0 ")

co.count()

0

In [52]:
co=sqlContext.sql("select max(calc_date) from om_out_df")

co.show()

+--------------+
|max(calc_date)|
+--------------+
|    2019-03-07|
+--------------+



In [54]:
co=sqlContext.sql("select OM_SCORE_REL_BUCKET,count(CALC_ID) from om_out_df where calc_date == '2019-03-07' GROUP BY OM_SCORE_REL_BUCKET ")

co.show()

+-------------------+--------------+
|OM_SCORE_REL_BUCKET|count(CALC_ID)|
+-------------------+--------------+
|               High|         13525|
|                Low|          7694|
|             Middle|         11091|
|            Decline|             8|
+-------------------+--------------+

